## Assignment: Student Management Database in PostgreSQL
Objective
In this exercise, you will connect to a PostgreSQL database you created, build the tables for a simple student management system, insert sample data, and perform different queries — including JOIN operations.

Part 1: Database Connection
Create a new database in PostgreSQL named student_management.

Write Python code (using psycopg2) to connect to your student_management database.

Example:

In [ ]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="Std#040228",
    host="localhost",
    # port="5432"
)

cur = conn.cursor()
print("Database connected successfully.")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE student_management;")



Database connected successfully.


## Part 2: Create Tables

Create the following tables using SQL:

### `students`
- `student_id` (SERIAL PRIMARY KEY)  
- `name` (VARCHAR)  
- `email` (VARCHAR)  
- `phone` (VARCHAR)  

### `courses`
- `course_id` (SERIAL PRIMARY KEY)  
- `course_name` (VARCHAR)  
- `credits` (INT)  

### `enrollments`
- `enrollment_id` (SERIAL PRIMARY KEY)  
- `student_id` (INT REFERENCES students(student_id))  
- `course_id` (INT REFERENCES courses(course_id))  
- `grade` (VARCHAR)  


In [6]:
cur.execute(" CREATE TABLE students (student_id serial primary key,name varchar(50), email varchar(255),phone varchar(20) )")
cur.execute(" CREATE TABLE courses (course_id serial primary key,course_name varchar(50), credits int )")
cur.execute(" CREATE TABLE enrollments (enrollment_id serial primary key,student_id int references students(student_id), course_id int references courses(course_id),grade varchar(20) )")
print("tables created")

tables created


## Part 3: Insert Sample Data
Insert at least 3 records into each table.

Example:

In [17]:
sql1 = "INSERT INTO students (name, email, phone) VALUES (%s,%s,%s)"
data1 =[
    ('Alice Johnson', 'alice@example.com', '1234567890'),
    ('Bob Smith', 'bob@example.com', '9876543210'),
    ('Charlie Brown', 'charlie@example.com', '5555555555'),
    ('Mohamed Ali','mohamedali@gmail.com','01020304050')
    ]
cur.executemany(sql1, data1)
conn.commit()

sql2 = "INSERT INTO courses (course_name, credits) VALUES (%s,%s)"
data2 =[
    ('Mathematics', 3),
    ('Computer Science', 4),
    ('History', 2)
    ]
cur.executemany(sql2, data2)
conn.commit()

sql3 = "INSERT INTO enrollments (student_id, course_id, grade) VALUES (%s,%s,%s)"
data3 =[
    (1, 1, 'A'),
    (1, 2, 'B'),
    (2, 3, 'A'),
    (3, 2, 'C')
    ]
cur.executemany(sql3, data3)
conn.commit()
print(cur.rowcount,"the new record was inserted")

4 the new record was inserted


## Part 4: Query Tasks

Write SQL queries to answer the following questions:

1. **List all students.**
   ```sql
   SELECT * FROM students;


In [2]:
cur.execute("SELECT  * FROM students")
students = cur.fetchall()

2. Find students who have a grade of 'A'.
Hint: Use a WHERE clause in the enrollments table to filter by grade.

SELECT * 
FROM enrollments
WHERE grade = 'A';


In [3]:
cur.execute("SELECT distinct * FROM students")
students = cur.fetchall()
print("Student information: ")
for i in students:
    print(i)
cur.execute("""
    SELECT s.name, e.course_id, e.grade
    FROM enrollments e
    JOIN students s ON e.student_id = s.student_id
    WHERE e.grade = 'A'
""")
print("superior students:\n",cur.fetchall())

Student information: 
(4, 'Mohamed Ali', 'mohamedali@gmail.com', '01020304050')
(2, 'Bob Smith', 'bob@example.com', '9876543210')
(3, 'Charlie Brown', 'charlie@example.com', '5555555555')
(1, 'Alice Johnson', 'alice@example.com', '1234567890')
superior students:
 [('Alice Johnson', 1, 'A'), ('Bob Smith', 3, 'A')]


3. Show all courses with their credit hours.
Hint: Select course name and credit hours from the courses table.


SELECT course_name, credit_hours 
FROM courses;

In [21]:
cur.execute("SELECT course_name, credits FROM courses")
results = cur.fetchall()
for i in results:
    print(i)

('Mathematics', 3)
('Computer Science', 4)
('History', 2)


4. Find the courses a specific student (e.g., 'Alice Johnson') is enrolled in.
Hint: Use JOIN between students, enrollments, and courses.


SELECT s.name, c.course_name
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
JOIN courses c ON e.course_id = c.course_id
WHERE s.name = 'Alice Johnson';

In [4]:
cur.execute("""
    SELECT s.name, c.course_id, c.course_name
    FROM students s
    JOIN enrollments e ON s.student_id = e.student_id
    JOIN courses c ON e.course_id = c.course_id
    WHERE s.name = 'Alice Johnson'
""")
print(cur.fetchall())

[('Alice Johnson', 1, 'Mathematics'), ('Alice Johnson', 2, 'Computer Science')]


5. List each student along with the number of courses they are enrolled in.
Hint: Use JOIN, GROUP BY, and COUNT().


SELECT s.name, COUNT(e.course_id) AS total_courses
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
GROUP BY s.name;

In [26]:
cur.execute("""
    SELECT s.name,COUNT(e.course_id) AS total_courses
    FROM students s
    JOIN enrollments e ON s.student_id = e.student_id GROUP BY s.name;
""")
print("students and number of enrolled courses:\n",cur.fetchall())

students and number of enrolled courses:
 [('Alice Johnson', 2), ('Bob Smith', 1), ('Charlie Brown', 1)]


6. Find students who are not enrolled in any course.
Hint: Use LEFT JOIN and check for NULL in the enrollments table.


SELECT s.name
FROM students s
LEFT JOIN enrollments e ON s.student_id = e.student_id
WHERE e.course_id IS NULL;


In [25]:
cur.execute("""
    SELECT s.name  
    FROM students s LEFT
    JOIN enrollments e ON s.student_id = e.student_id 
    WHERE e.course_id IS NULL
""")
print("Students who are not enrolled in any courses:\n",cur.fetchall())

Students who are not enrolled in any courses:
 [('Mohamed Ali',)]


7. Show all enrollments sorted by grade (highest to lowest).
Hint: Use ORDER BY with DESC.


SELECT * 
FROM enrollments
ORDER BY grade DESC;



In [ ]:
cur.execute("""
    SELECT s.name, e.course_id, e.grade 
    FROM enrollments e 
    JOIN students s ON e.student_id = s.student_id
    ORDER BY grade DESC;
""")
print(cur.fetchall())

[('Charlie Brown', 2, 'C'), ('Alice Johnson', 2, 'B'), ('Alice Johnson', 1, 'A'), ('Bob Smith', 3, 'A')]


Part 5: Bonus
Write a query to find the average grade per course (Convert grades into numeric values if you want to calculate an average).

Add a instructor table and modify your queries to include instructor names.

